In [1]:
import os
import wfdb
from collections import Counter
import pandas as pd
import numpy as np

In [7]:
ecg_data = os.path.join(os.getcwd(), '/Users/matildacotton/Desktop/semester_2_2024/topics/data/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/WFDBRecords')

hea_files = []
mat_files = []

for root, dirs, files in os.walk(ecg_data):
    for file in files:
        if file.endswith('.hea'):
            hea_files.append(os.path.join(root, file))
        elif file.endswith('.mat'):
            mat_files.append(os.path.join(root, file))


print('\nTotal HEA Files:', len(hea_files))
print('Total MAT Files:', len(mat_files))


Total HEA Files: 45152
Total MAT Files: 45152


In [9]:
ecg_data = os.path.join(os.getcwd(), '/Users/matildacotton/Desktop/semester_2_2024/topics/data/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/WFDBRecords')

conditions = []

mapping_file = os.path.join(os.getcwd(), '/Users/matildacotton/Desktop/semester_2_2024/topics/data/a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0/ConditionNames_SNOMED-CT.csv')
mapping_df = pd.read_csv(mapping_file)
mapping_dict = dict(zip(mapping_df['Snomed_CT'], mapping_df['Full Name']))


for hea_file in hea_files:
    base_name = os.path.splitext(os.path.basename(hea_file))[0]
    
    header_path = os.path.join(os.path.dirname(hea_file), base_name)

    record = wfdb.rdheader(header_path)
    
    try:
        record = wfdb.rdheader(header_path)


        #extracting comments
        comments = record.comments[2].split(':')
        b = comments[1]
        b = b.split(",")
        diagnosis_code = []
        for x in b:
            diagnosis_code.append(int(x))


        code = ''
        
        for x in diagnosis_code:
            diagnosis_description = mapping_dict.get(x, 'Unknown')
            code += diagnosis_description
        
        conditions.append([base_name,diagnosis_description])

    except FileNotFoundError:
        print(f'File not found: {hea_file}')
    except ValueError as e:
        print(f"ValueError while reading {hea_file}: {e}")
        continue

ValueError: time data '/' does not match format '%d/%m/%Y'

In [5]:
conditions = np.array(conditions)

df = pd.DataFrame(conditions)

df.to_csv('files_and_conditions_fixed.csv', index=False, header=False)
